# Tracking AR 12970 and creating an ndcube

In this example we will attempt to find AR 12790, download HMI and AIA 193 data for it, track the AR in carrington coordinates and then create an animation of a composite image.


To install ndcube run: `pip install "ndcube>=2.0.0b1"`

In [53]:
import copy

import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.wcs import WCS
from reproject import reproject_adaptive, reproject_interp

import sunpy.map
import sunpy.coordinates
from sunpy.net import Fido, attrs as a, hek

from ndcube import NDCube

## Obtaining the Data

Let's start by finding a reference coord for AR 12790

In [277]:
hclient = hek.HEKClient()

In [278]:
res = hclient.search(hek.attrs.Time("2020/11/01", "2020/12/17"),
                     hek.attrs.AR,
                     hek.attrs.AR.NOAANum == 12790)

In [279]:
res['ar_noaanum', 'event_starttime', 'hgc_coord']

ar_noaanum,event_starttime,hgc_coord
int64,str19,str21
12790,2020-12-02T00:00:00,POINT(-106.60512 -23)
12790,2020-12-03T00:00:00,POINT(-107.7835 -23)
12790,2020-12-04T00:00:00,POINT(-106.96162 -22)
12790,2020-12-05T00:00:00,POINT(-108.13947 -23)
12790,2020-12-06T00:00:00,POINT(-109.31706 -23)
12790,2020-12-07T00:00:00,POINT(-108.49438 -23)
12790,2020-12-08T00:00:00,POINT(-108.67141 -22)
12790,2020-12-09T00:00:00,POINT(-107.84819 -21)
12790,2020-12-10T00:00:00,POINT(-108.0247 -23)


In [280]:
res['hgc_coord'] = SkyCoord(np.float64([c.strip("POINT()").split() for c in res['hgc_coord']]),
                            unit=u.deg,
                            frame="heliographic_carrington")

In [281]:
res['event_starttime'] = Time(res['event_starttime'])

In [282]:
ar12790 = res['ar_noaanum', 'event_starttime', 'hgc_coord']

In [283]:
ar12790

ar_noaanum,event_starttime,hgc_coord
,,"deg,deg,km"
int64,object,object
12790,2020-12-02T00:00:00.000,"253.39488,-23.0,695700.0"
12790,2020-12-03T00:00:00.000,"252.2165,-23.0,695700.0"
12790,2020-12-04T00:00:00.000,"253.03838000000002,-22.0,695700.0"
12790,2020-12-05T00:00:00.000,"251.86052999999998,-23.0,695700.0"
12790,2020-12-06T00:00:00.000,"250.68294,-23.0,695700.0"
12790,2020-12-07T00:00:00.000,"251.50562,-23.0,695700.0"
12790,2020-12-08T00:00:00.000,"251.32859000000002,-22.0,695700.0"
12790,2020-12-09T00:00:00.000,"252.15181,-21.0,695700.0"


In [9]:
start_time, end_time = ar12790['event_starttime'][[0,-1]]

In [10]:
start_time, end_time

(<Time object: scale='utc' format='isot' value=2020-12-02T00:00:00.000>,
 <Time object: scale='utc' format='isot' value=2020-12-13T00:00:00.000>)

In [11]:
hmi = a.Instrument.hmi & a.Physobs.los_magnetic_field
aia = a.Instrument.aia & a.Wavelength(19.3 * u.nm)

In [246]:
results = Fido.search(a.Time(start_time, end_time), a.Sample(6*u.hour),
                      hmi | aia)

In [247]:
results

Start Time [1],End Time [1],Source,Instrument,Type,Wavelength [2]
,,,,,Angstrom
str19,str19,str3,str3,str8,float64
2020-12-02 00:00:23,2020-12-02 00:00:24,SDO,HMI,FULLDISK,6173.0 .. 6174.0
2020-12-02 06:00:23,2020-12-02 06:00:24,SDO,HMI,FULLDISK,6173.0 .. 6174.0
2020-12-02 12:00:23,2020-12-02 12:00:24,SDO,HMI,FULLDISK,6173.0 .. 6174.0
2020-12-02 18:00:23,2020-12-02 18:00:24,SDO,HMI,FULLDISK,6173.0 .. 6174.0
2020-12-03 00:00:23,2020-12-03 00:00:24,SDO,HMI,FULLDISK,6173.0 .. 6174.0
2020-12-03 06:00:23,2020-12-03 06:00:24,SDO,HMI,FULLDISK,6173.0 .. 6174.0
2020-12-03 12:00:23,2020-12-03 12:00:24,SDO,HMI,FULLDISK,6173.0 .. 6174.0
2020-12-03 18:00:23,2020-12-03 18:00:24,SDO,HMI,FULLDISK,6173.0 .. 6174.0


In [248]:
files = Fido.fetch(results, path="~/psi_data/12790/{instrument}/")

In [259]:
files

['/home/stuart/psi_data/12790/HMI/hmi_m_45s_2020_12_07_00_01_30_tai_magnetogram.fits', '/home/stuart/psi_data/12790/HMI/hmi_m_45s_2020_12_10_00_01_30_tai_magnetogram.fits', '/home/stuart/psi_data/12790/AIA/aia_lev1_193a_2020_12_04t06_00_04_84z_image_lev1.fits', '/home/stuart/psi_data/12790/HMI/hmi_m_45s_2020_12_08_00_01_30_tai_magnetogram.fits', '/home/stuart/psi_data/12790/HMI/hmi_m_45s_2020_12_11_06_01_30_tai_magnetogram.fits', '/home/stuart/psi_data/12790/AIA/aia_lev1_193a_2020_12_04t18_00_04_84z_image_lev1.fits', '/home/stuart/psi_data/12790/AIA/aia_lev1_193a_2020_12_08t18_00_04_84z_image_lev1.fits', '/home/stuart/psi_data/12790/HMI/hmi_m_45s_2020_12_06_06_01_30_tai_magnetogram.fits', '/home/stuart/psi_data/12790/HMI/hmi_m_45s_2020_12_09_00_01_30_tai_magnetogram.fits', '/home/stuart/psi_data/12790/AIA/aia_lev1_193a_2020_12_09t12_00_04_84z_image_lev1.fits', '/home/stuart/psi_data/12790/AIA/aia_lev1_193a_2020_12_09t18_00_04_84z_image_lev1.fits', '/home/stuart/psi_data/12790/HMI/hmi_m

## Processing the Data

Now we have got the data let's take the HMI images and reproject them into carrington coordinates so we can clearly see the evolution of the feature.

In [250]:
%matplotlib

Using matplotlib backend: module://ipympl.backend_nbagg


In [287]:
ar12790 = ar12790[:11]

In [263]:
hmi_maps = sunpy.map.Map("~/psi_data/12790/HMI/*")
#aia_maps = sunpy.map.Map("~/psi/12790/AIA/*") 

In [284]:
data = [[h.observer_coordinate.lon, h.observer_coordinate.lat, h.observer_coordinate.radius] for h in hmi_maps[::4]]

In [285]:
observers = SkyCoord(data, unit=(u.deg, u.deg, u.m), obstime=[h.date for h in hmi_maps[::4]], frame="heliographic_stonyhurst")

In [288]:
ar12790['hgc_coord'] = SkyCoord(ar12790['hgc_coord'], observer=observers, obstime=[h.date for h in hmi_maps[::4]])

In [289]:
ar12790['hgc_coord']

<SkyCoord (HeliographicCarrington: obstime=['2020-12-02T00:00:23.400' '2020-12-03T00:00:23.300'
 '2020-12-04T00:00:23.300' '2020-12-05T00:00:23.200'
 '2020-12-06T00:00:23.100' '2020-12-07T00:00:23.100'
 '2020-12-08T00:00:23.000' '2020-12-09T00:00:22.900'
 '2020-12-10T00:00:22.900' '2020-12-11T00:00:22.800'
 '2020-12-12T00:00:22.800'], observer=<HeliographicStonyhurst Coordinate (obstime=['2020-12-02T00:00:23.400' '2020-12-03T00:00:23.300'
 '2020-12-04T00:00:23.300' '2020-12-05T00:00:23.200'
 '2020-12-06T00:00:23.100' '2020-12-07T00:00:23.100'
 '2020-12-08T00:00:23.000' '2020-12-09T00:00:22.900'
 '2020-12-10T00:00:22.900' '2020-12-11T00:00:22.800'
 '2020-12-12T00:00:22.800']): (lon, lat, radius) in (deg, deg, m)
    [(-0.01912113,  0.727277, 1.47471968e+11),
     (-0.01910594,  0.599633, 1.47448999e+11),
     (-0.0190744 ,  0.471764, 1.47426850e+11),
     (-0.01907004,  0.343709, 1.47405508e+11),
     (-0.01906129,  0.21551 , 1.47384953e+11),
     (-0.01901451,  0.087205, 1.47365159e+11

In [290]:
plt.figure()
ax = plt.subplot(projection=hmi_maps[0])
hmi_maps[0].plot(vmin=-1500, vmax=1500, cmap='hmimag')
ax.plot_coord(ar12790['hgc_coord'], "wo")
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
/home/stuart/.virtualenvs/predsci/lib/python3.9/site-packages/astropy/visualization/wcsaxes/core.py:211: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  return super().imshow(X, *args, origin=origin, **kwargs)


In [291]:
def make_carrington_wcses(maps, lat_scale, lon_scale):
    centre_lat = ar12790['hgc_coord'][0].lat
    scale = [1/lon_scale, 1/lat_scale]*u.deg/u.pix
    shape = (40*lat_scale, 40*lon_scale)
    
    lat_pix_offset = -centre_lat / scale[1]
    
    wcses = []
    for amap in maps:
        observer = amap.observer_coordinate
        header = sunpy.map.make_fitswcs_header(
            shape,
            SkyCoord(ar12790['hgc_coord'][0].lon, 0,
                     unit=u.deg, observer=observer, obstime=amap.date, frame="heliographic_carrington"),
            scale=scale,
            reference_pixel=u.Quantity([(shape[1]-1)/2*u.pix, ((shape[0]-1)/2*u.pix) + lat_pix_offset]),
            projection_code="CAR"
        )

        wcs = WCS(header)
        wcs.heliographic_observer = observer
        
        wcses.append(wcs)
        
    return shape, wcses

In [292]:
shape, hmi_carrington_wcses = make_carrington_wcses(hmi_maps, 35, 35)

In [294]:
%matplotlib widget

In [314]:
out_maps = []
for in_map, out_wcs in tqdm(zip(hmi_maps, hmi_carrington_wcses), total=len(hmi_maps)):
    array, _ = reproject_adaptive(in_map, out_wcs, shape_out=shape)
    out_maps.append(sunpy.map.Map(array, out_wcs))

a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating-point value was expected. [astropy.wcs.wcs]
a floating

## Making a NDCube

Now we have the data on a shared grid, let's build a NDCube out of the results, with a 3D WCS and then experiment a little with what NDCube can provide.

In [315]:
seq = sunpy.map.Map(out_maps, sequence=True)

In [316]:
times = Time([m.date for m in seq])
time_delta = times[1:] - times[:-1]
time_delta.to(u.hour).round()

<Quantity [6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
           6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
           6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.] h>

In [317]:
cube_header = copy.copy(seq[0].meta)

In [318]:
cube_header['wcsaxes'] = 3
cube_header['naxis'] = 3
cube_header['CTYPE3'] = "UTC"
cube_header['CDELT3'] = time_delta[0].to_value(u.s)
cube_header['CUNIT3'] = "s"
cube_header['CRVAL3'] = 0
cube_header['CRPIX3'] = 0
cube_header['DATEREF'] = times[0].isot
del cube_header['mjdref']
del cube_header['mjd-obs']

In [319]:
cube_wcs = WCS(cube_header)

In [320]:
ndc = NDCube(np.rollaxis(seq.as_array(), -1), wcs=cube_wcs)

In [321]:
ndc.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
ndc.array_axis_physical_types

[('time',),
 ('custom:pos.heliographic.carrington.lon',
  'custom:pos.heliographic.carrington.lat'),
 ('custom:pos.heliographic.carrington.lon',
  'custom:pos.heliographic.carrington.lat')]

In [76]:
small_cube = ndc.crop_by_values([240, -33, None], [260, -13, None], units=(u.deg, u.deg, u.s))

In [51]:
small_cube.plot(vmin=-1500, vmax=1500)

In [324]:
plt.figure()
ndc[:,650,:].plot(aspect="auto")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<WCSAxesSubplot:>

In [ ]:
small_cube.axis_world_coords()

In [ ]:
small_cube.axis_world_coords("time")

In [ ]:
h = hmi_maps[0]

In [ ]:
h.draw_contours

In [ ]:
plt.figure()
ax = plt.subplot(projection=hmi_maps[0])
#aia_maps[0].plot()
hmi_maps[0].draw_contours(levels=np.linspace(hm, 100, 100)*u.percent, axes=ax)